In [1]:
from spark_batch.lib.spark_session import get_spark_session
from spark_batch.lib.elt_manager import EltManager
from spark_batch.lib.pxlogger import CustomLogger
from spark_batch.lib.util import toTargetCondition
import logging
import os 

CustomLogger.set_custom_log_level(logging.DEBUG)  #default log_level is 'INFO'
spark = get_spark_session(app_name="f_load_subway_passengers_time",notebook_name="test", notebook_namespace="demo01-kf", \
            executor_instances="1", executor_memory="1G", executor_cores="1", driver_memory="1G", driver_cores="1" )
em = EltManager(spark, config_file="config.yaml", domain="life", record_type="mart", record_prefix="hopt")

:: loading settings :: url = jar:file:/usr/local/spark-3.5.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f43b470f-32e1-44ad-92af-3d3f0af63b95;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 i

2025-07-01 02:55:48,787 - DEBUG - ResourceManager:021 - ('config keys=', dict_keys(['postgresql-life', 'csv1', 'json1', 'delta1', 'mart']))


### JSON > Bronze

In [2]:
source_type = "json1";  source_topic = "paasup"; source_dpath="datasource/life"
target_type = "delta1";   target_topic = "paasup"; target_dpath="bronze/life"

em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 500000)

2025-07-01 02:55:48,794 - DEBUG - ResourceManager:047 - s3_ssl_verify is False
2025-07-01 02:55:49,790 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'bronze/life')


In [8]:
def s3_file_exists(spark_context, s3_path):
    """
    주어진 S3 경로에 파일이 존재하는지 확인합니다.

    Args:
        spark_context: 활성 SparkContext.
        s3_path (str): 확인할 S3 파일 경로 (예: "s3a://버킷/경로/파일.json").

    Returns:
        bool: 파일이 존재하면 True, 그렇지 않으면 False.
    """
    try:
        # Hadoop Path 객체 생성
        hadoop_path = spark_context._jvm.org.apache.hadoop.fs.Path(s3_path)
        # 파일 시스템 객체 가져오기
        fs = hadoop_path.getFileSystem(spark_context._jsc.hadoopConfiguration())
        # 파일 존재 여부 확인
        return fs.exists(hadoop_path)
    except Exception as e:
        print(f"S3 파일 '{s3_path}' 존재 여부 확인 중 오류: {e}")
        return False
        
#print(s3_file_exists(spark.sparkContext, "s3a://paasup/datasource/life/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.json"))
print(s3_file_exists(spark.sparkContext, "s3a://paasup/datasource/life/subway_time.json"))


True


In [3]:
#df = spark.read.json("s3a://paasup/datasource/life/subway_time.json")
df = spark.read.json("s3a://paasup/datasource/life/subway_passengers_time.json")
df.printSchema()

25/07/01 02:56:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/07/01 02:56:01 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


root
 |-- _corrupt_record: string (nullable = true)
 |-- hr_0_get_off_nope: long (nullable = true)
 |-- hr_0_get_on_nope: long (nullable = true)
 |-- hr_10_get_off_nope: long (nullable = true)
 |-- hr_10_get_on_nope: long (nullable = true)
 |-- hr_11_get_off_nope: long (nullable = true)
 |-- hr_11_get_on_nope: long (nullable = true)
 |-- hr_12_get_off_nope: long (nullable = true)
 |-- hr_12_get_on_nope: long (nullable = true)
 |-- hr_13_get_off_nope: long (nullable = true)
 |-- hr_13_get_on_nope: long (nullable = true)
 |-- hr_14_get_off_nope: long (nullable = true)
 |-- hr_14_get_on_nope: long (nullable = true)
 |-- hr_15_get_off_nope: long (nullable = true)
 |-- hr_15_get_on_nope: long (nullable = true)
 |-- hr_16_get_off_nope: long (nullable = true)
 |-- hr_16_get_on_nope: long (nullable = true)
 |-- hr_17_get_off_nope: long (nullable = true)
 |-- hr_17_get_on_nope: long (nullable = true)
 |-- hr_18_get_off_nope: long (nullable = true)
 |-- hr_18_get_on_nope: long (nullable = true)


In [4]:
df.createOrReplaceTempView("my_table_name")
df.show()

25/07/01 02:56:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+-----------------+----------------+--------+---------------+----------------------+------+
|     _corrupt_record|hr_0_get_off_nope|hr_0_get_on_no

In [5]:
source_objects = ['subway_passengers_time.json']; target_object = "SUBWAY_PASSENGERS_TIME"
#source_tm = em.getSourceManager()
#source_df = source_tm.loadTable(source_objects[0])

#source_inc_query = f"""
#select * from SUBWAY_INFO
#""" 

source_df, target_df, valid = em.ingest_full(source_objects, target_object)

2025-07-01 02:56:26,566 - INFO - EltManager:194 - ETL/FL Started : [ delta1 paasup bronze/life SUBWAY_PASSENGERS_TIME (['subway_passengers_time.json']) ]
2025-07-01 02:56:26,568 - DEBUG - JSONTableManager:040 - ('loadTable = ', 'subway_passengers_time.json', None, None)
2025-07-01 02:56:26,569 - DEBUG - JSONTableManager:043 - ('file_path = ', 's3a://paasup/datasource/life/subway_passengers_time.json')
2025-07-01 02:56:26,569 - DEBUG - JSONTableManager:048 - ('subway_passengers_time.json', True, True, None, None, None)


2025-07-01 02:56:27,948 - DEBUG - JSONTableManager:055 - ('TempView = ', 'subway_passengers')


2025-07-01 02:56:29,500 - INFO - EltManager:226 - Source count = 74,282 / expected loop 1 / 2.93
2025-07-01 02:56:29,500 - DEBUG - JSONTableManager:040 - ('loadTable = ', 'subway_passengers_time.json', None, None)
2025-07-01 02:56:29,501 - DEBUG - JSONTableManager:043 - ('file_path = ', 's3a://paasup/datasource/life/subway_passengers_time.json')
2025-07-01 02:56:29,501 - DEBUG - JSONTableManager:048 - ('subway_passengers_time.json', True, True, None, None, None)


2025-07-01 02:56:30,884 - DEBUG - JSONTableManager:055 - ('TempView = ', 'subway_passengers')
2025-07-01 02:56:30,943 - DEBUG - DeltaTableManager:099 - ('saveTable = ', 'SUBWAY_PASSENGERS_TIME', None, None, 'overwrite', False, None)


25/07/01 02:56:50 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


...

2025-07-01 02:58:05,008 - DEBUG - DeltaTableManager:084 - ('dt_with_row size & offset = ', 74282, None)


2025-07-01 02:58:06,234 - INFO - EltManager:284 - Target Saving Count : delta1 paasup bronze/life SUBWAY_PASSENGERS_TIME (74,282)


2025-07-01 02:58:08,708 - INFO - EltManager:288 - ETL/FL Done : [ delta1 paasup bronze/life SUBWAY_PASSENGERS_TIME / True (74,282, 74,282, 0) / 102.14 ]


2025-07-01 02:58:10,016 - DEBUG - EltManager:115 - self.record is True
2025-07-01 02:58:10,123 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:58:10,124 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:58:10,124 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:58:10,125 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 02:58:12,301 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:58:12,302 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:58:12,302 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:58:12,440 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 9643, after = 9644 [ public.elt_manager_log ]


### Bronze > Gold

In [7]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="bronze/life"
target_type = "delta1"; target_topic = "paasup";  target_dpath="gold"
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 50000000)

2025-07-01 02:58:12,538 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'bronze/life')
2025-07-01 02:58:12,547 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'gold')


In [8]:
source_objects = ["SUBWAY_PASSENGERS_TIME"];  target_object = "subway_passengers_time"

source_inc_query = f"""
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '00시-01시' time_term
                             , hr_0_get_on_nope pass_in
                             , hr_0_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '01시-02시' time_term
                             , hr_1_get_on_nope pass_in
                             , hr_1_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '02시-03시' time_term
                             , hr_2_get_on_nope pass_in
                             , hr_2_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '03시-04시' time_term
                             , hr_3_get_on_nope pass_in
                             , hr_3_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '04시-05시' time_term
                             , hr_4_get_on_nope pass_in
                             , hr_4_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '05시-06시' time_term
                             , hr_5_get_on_nope pass_in
                             , hr_5_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '06시-07시' time_term
                             , hr_6_get_on_nope pass_in
                             , hr_6_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '07시-08시' time_term
                             , hr_7_get_on_nope pass_in
                             , hr_7_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '08시-09시' time_term
                             , hr_8_get_on_nope pass_in
                             , hr_8_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '09시-10시' time_term
                             , hr_9_get_on_nope pass_in
                             , hr_9_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '10시-11시' time_term
                             , hr_10_get_on_nope pass_in
                             , hr_10_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '11시-12시' time_term
                             , hr_11_get_on_nope pass_in
                             , hr_11_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '12시-13시' time_term
                             , hr_12_get_on_nope pass_in
                             , hr_12_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '13시-14시' time_term
                             , hr_13_get_on_nope pass_in
                             , hr_13_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '14시-15시' time_term
                             , hr_14_get_on_nope pass_in
                             , hr_14_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '15시-16시' time_term
                             , hr_15_get_on_nope pass_in
                             , hr_15_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '16시-17시' time_term
                             , hr_16_get_on_nope pass_in
                             , hr_16_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '17시-18시' time_term
                             , hr_17_get_on_nope pass_in
                             , hr_17_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '18시-19시' time_term
                             , hr_18_get_on_nope pass_in
                             , hr_18_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '19시-20시' time_term
                             , hr_19_get_on_nope pass_in
                             , hr_19_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '20시-21시' time_term
                             , hr_20_get_on_nope pass_in
                             , hr_20_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '21시-22시' time_term
                             , hr_21_get_on_nope pass_in
                             , hr_21_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '22시-23시' time_term
                             , hr_22_get_on_nope pass_in
                             , hr_22_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
                        union all 
                        select use_mm use_ym
                             , sbwy_rout_ln_nm line_no
                             , sttn station_name
                             , '23시-24시' time_term
                             , hr_23_get_on_nope pass_in
                             , hr_23_get_off_nope pass_out
                             , job_ymd reg_date
                          from SUBWAY_PASSENGERS_TIME
"""
# 전부 삭제 
target_condition = f"""
 1=1
"""
source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 02:58:12,597 - INFO - EltManager:334 - ETL/IC Started : [ delta1 paasup gold subway_passengers_time (['SUBWAY_PASSENGERS_TIME']) ]
2025-07-01 02:58:12,598 - DEBUG - DeltaTableManager:155 - ('queryTable = ', "\n                        select use_mm use_ym\n                             , sbwy_rout_ln_nm line_no\n                             , sttn station_name\n                             , '00시-01시' time_term\n                             , hr_0_get_on_nope pass_in\n                             , hr_0_get_off_nope pass_out\n                             , job_ymd reg_date\n                          from SUBWAY_PASSENGERS_TIME\n                        union all \n                        select use_mm use_ym\n                             , sbwy_rout_ln_nm line_no\n                             , sttn station_name\n                             , '01시-02시' time_term\n                             , hr_1_get_on_nope pass_in\n                             , hr_1_get_off_nope pass_out\

25/07/01 02:58:20 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.
25/07/01 02:58:56 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


2025-07-01 02:58:59,855 - DEBUG - DeltaTableManager:200 - delSert : before = 1767864, after = 1782768, del = 1767864 [ s3a://paasup/gold/subway_passengers_time / 
 1=1
 ]


2025-07-01 02:59:00,400 - DEBUG - DeltaTableManager:338 - 
        version       operation  numDeletedRows   numOutputRows                     operationParameters
              2           WRITE            None         1782768 {'mode': 'Append', 'partitionBy': '[]'}
              1          DELETE         1767864            None               {'predicate': '["true"]'}


2025-07-01 02:59:03,761 - INFO - EltManager:369 - Source Loading : delta1 paasup bronze/life ['SUBWAY_PASSENGERS_TIME'] / source_size=1,782,768 / elipsed=51.16
2025-07-01 02:59:03,762 - INFO - EltManager:371 - Target  Saving : delta1 paasup gold subway_passengers_time / delsert_size=1,782,768 (before=1,767,864, after=1,782,768, del=1,767,864) / elipsed=0.00


2025-07-01 02:59:04,723 - DEBUG - DeltaTableManager:284 - ('target_table=', 's3a://paasup/gold/subway_passengers_time', ', condition= ', '\n 1=1\n', ', count=', 1782768)
2025-07-01 02:59:04,724 - INFO - EltManager:375 - ETL/IC Done : [ delta1 paasup gold subway_passengers_time / True (1,782,768, 1,782,768, 0) / 52.13 ]
2025-07-01 02:59:04,725 - DEBUG - EltManager:115 - self.record is True
2025-07-01 02:59:04,762 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:59:04,763 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:59:04,763 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:59:04,764 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.

2025-07-01 02:59:05,239 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 02:59:05,240 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 02:59:05,240 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:59:05,345 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 9644, after = 9645 [ public.elt_manager_log ]


### Gold > MART

In [9]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="gold"
target_type = "mart"; target_topic = "postgres";  target_dpath=None
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 500000)

2025-07-01 02:59:05,349 - DEBUG - ResourceManager:052 - ('source_type_name = delata  :', 'paasup', 'gold')


In [10]:
source_objects = ["subway_passengers_time"];  target_object = "subway_passengers_time"
source_inc_query = f"""
                        select * from subway_passengers_time
                        where use_ym is not null
                    """

# 전부 삭제 
target_condition = f"""
 1=1
"""
source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 02:59:05,369 - INFO - EltManager:334 - ETL/IC Started : [ mart postgres _ subway_passengers_time (['subway_passengers_time']) ]
2025-07-01 02:59:05,371 - DEBUG - DeltaTableManager:155 - ('queryTable = ', '\n                        select * from subway_passengers_time\n                        where use_ym is not null\n                    ', ['subway_passengers_time'], None, None, None)
2025-07-01 02:59:05,372 - DEBUG - DeltaTableManager:127 - ('loadTables = ', ['subway_passengers_time'], None, None, None)
2025-07-01 02:59:05,442 - DEBUG - PostgreSQLTableManager:397 - ('delSert = ', '\n 1=1\n', 'subway_passengers_time')
2025-07-01 02:59:05,442 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_passengers_time', None, 100000)
2025-07-01 02:59:05,443 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_passengers_time')
2025-07-01 02:59:05,444 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgre

2025-07-01 02:59:11,884 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'subway_passengers_time', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 02:59:11,885 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_passengers_time', None, 100000)
2025-07-01 02:59:11,885 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_passengers_time')
2025-07-01 02:59:11,886 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:35,195 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_passengers_time', None, 100000)
2025-07-01 03:02:35,196 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_passengers_time')
2025-07-01 03:02:35,197 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:36,128 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 0, after = 1782648 [ postgres.public.subway_passengers_time ]
2025-07-01 03:02:36,129 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_passengers_time', None, 100000)
2025-07-01 03:02:36,129 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_passengers_time')
2025-07-01 03:02:36,130 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:36,958 - DEBUG - PostgreSQLTableManager:421 - delSert : before = 1767744, after = 1782648, del = 1767744 [ postgres.public.subway_passengers_time / 
 1=1
 ]


2025-07-01 03:02:39,570 - INFO - EltManager:369 - Source Loading : delta1 paasup gold ['subway_passengers_time'] / source_size=1,782,648 / elipsed=214.20
2025-07-01 03:02:39,571 - INFO - EltManager:371 - Target  Saving : mart postgres _ subway_passengers_time / delsert_size=1,782,648 (before=1,767,744, after=1,782,648, del=1,767,744) / elipsed=0.00
2025-07-01 03:02:39,571 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'subway_passengers_time', None, 100000)
2025-07-01 03:02:39,572 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'postgres.public.subway_passengers_time')
2025-07-01 03:02:39,573 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:40,822 - INFO - EltManager:375 - ETL/IC Done : [ mart postgres _ subway_passengers_time / True (1,782,648, 1,782,648, 0) / 215.45 ]
2025-07-01 03:02:40,823 - DEBUG - EltManager:115 - self.record is True
2025-07-01 03:02:40,858 - DEBUG - PostgreSQLTableManager:302 - ('saveTable = ', 'elt_manager_log', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')
2025-07-01 03:02:40,859 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 03:02:40,859 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 03:02:40,859 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:41,883 - DEBUG - PostgreSQLTableManager:269 - ('loadTable = ', 'elt_manager_log', None, 100000)
2025-07-01 03:02:41,884 - DEBUG - PostgreSQLTableManager:272 - ('target_table = ', 'public.elt_manager_log')
2025-07-01 03:02:41,885 - DEBUG - PostgreSQLTableManager:279 - ('connection_url = ', 'jdbc:postgresql://demo01-postgresql2-postgresql-ha-postgresql.demo01-postgresql2.svc.cluster.local:5432/postgres')


2025-07-01 03:02:42,214 - DEBUG - PostgreSQLTableManager:334 - saveTable : before = 9645, after = 9646 [ public.elt_manager_log ]


In [11]:
spark.stop()

25/07/01 03:02:52 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
